<a href="https://colab.research.google.com/github/AfriLANG4/masakhane/blob/master/jw300_bpe_en_xh_masakhane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Masakhane - Machine Translation for African Languages (Using JoeyNMT)

## Note before beginning:
### - The idea is that you should be able to make minimal changes to this in order to get SOME result for your own translation corpus. 

### - The tl;dr: Go to the **"TODO"** comments which will tell you what to update to get up and running

### - If you actually want to have a clue what you're doing, read the text and peek at the links

### - With 100 epochs, it should take around 7 hours to run in Google Colab

### - Once you've gotten a result for your language, please attach and email your notebook that generated it to masakhanetranslation@gmail.com

### - If you care enough and get a chance, doing a brief background on your language would be amazing. See examples in  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Retrieve your data & make a parallel corpus

If you are wanting to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "xh" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead!
!mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt-$tag"
os.environ["gdrive_path"] = "/content/drive/My Drive/masakhane/%s-%s-%s" % (source_language, target_language, tag)

In [0]:
!echo $gdrive_path

/content/drive/My Drive/masakhane/en-xh-baseline


In [0]:
# Install opus-tools
! pip install opustools-pkg

     |████████████████████████████████| 81kB 2.3MB/s 


In [0]:
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-xh.xml.gz not found. The following files are available for downloading:

   7 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en-xh.xml.gz
 263 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en.zip
  78 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/xh.zip

 348 MB Total size
./JW300_latest_xml_en-xh.xml.gz ... 100% of 7 MB
./JW300_latest_xml_en.zip ... 100% of 263 MB
./JW300_latest_xml_xh.zip ... 100% of 78 MB


In [0]:
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2019-11-28 02:54:46--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.05s   

2019-11-28 02:54:46 (5.39 MB/s) - ‘test.en-any.en’ saved [277791/277791]

--2019-11-28 02:54:49--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-xh.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Leng

In [0]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [0]:
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(3)

Loaded data and skipped 7158/876188 lines since contained in test set.


,source_sentence,target_sentence
0,How One Marriage Was Saved,Indlela Owasindiswa Ngayo Lo Mtshato
1,“ The application of the counsel in the book M...,Omnye umfundi onoxabiso nowaseMzantsi Afrika w...
2,"“ Chapter 5 , ‘ A Wife Who Is Dearly Loved , ’...","“ Isahluko 5 , esithi ‘ Umfazi Othandwa Kunene..."


In [0]:
df[df.duplicated()]

,source_sentence,target_sentence
202,But how ?,Kodwa njani ?
352,“ The application of the counsel in the book M...,Omnye umfundi onoxabiso nowaseMzantsi Afrika w...
353,"“ Chapter 5 , ‘ A Wife Who Is Dearly Loved , ’...","“ Isahluko 5 , esithi ‘ Umfazi Othandwa Kunene..."
354,I never imagined in my wildest dreams that I c...,Andizange ndicinge nasephupheni ukuba ndandise...
355,"Thank you very , very much .",Enkosi kakhulu .
...,...,...
868827,MEMORIAL ATTENDANCE ( 2015 ),ABEZE ESIKHUMBUZWENI ( 2015 )
868828,This publication is not for sale .,Le magazini ayithengiswa .
868857,Jesus said : “ ‘ You must love Jehovah your Go...,UYesu wathi : “ ‘ Uze umthande uYehova uThixo ...
868984,Why ?,Ngoba ?


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [0]:
print("Length of Data before Removing duplicate: ",len(df))
df = df.drop_duplicates()
print("Length of Data after Removing duplicate: ",len(df))

Length of Data before Removing duplicate:  869031
Length of Data after Removing duplicate:  817515


In [0]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df["source_sentence"] = df["source_sentence"].str.lower()
    df["target_sentence"] = df["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df.tail(num_dev_patterns) # Herman: Error in original
stripped = df.drop(df.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)


# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*
! head test.*

==> train.en <==
How One Marriage Was Saved
“ The application of the counsel in the book Making Your Family Life Happy saved my marriage , ” wrote an appreciative reader from South Africa .
“ Chapter 5 , ‘ A Wife Who Is Dearly Loved , ’ opened my eyes .
I never imagined in my wildest dreams that I could unwittingly cause so many problems .
Thank you very , very much .
My marriage had been in the most stormy parts of the sea , and now , after months , it is back in the quiet harbour of happiness . ”
If you desire help in making your marriage a happier one , you can benefit from this book .
Receive a copy by filling in and mailing the coupon .
Please send , postpaid , the hardcover , 192 - page book Making Your Family Life Happy .
I enclose $ 1 ( U.S . ) .

==> train.xh <==
Indlela Owasindiswa Ngayo Lo Mtshato
Omnye umfundi onoxabiso nowaseMzantsi Afrika wabhala wenjenje : “ Ukusebenzisa isiluleko esikwincwadi ethi Ukwenza Ubomi Bentsapho Yakho Bonwabe kwawusindisa umtshato wam . ”
“ Isa

In [0]:
! cat train.en | wc -l
! cat train.xh | wc -l
! cat dev.en | wc -l
! cat dev.xh | wc -l
! cat test.en | wc -l
! cat test.xh | wc -l

816515
816515
1000
1000
2717
2717




---


# Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [0]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Cloning into 'joeynmt'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 2199 (delta 4), reused 5 (delta 3), pack-reused 2184
Receiving objects: 100% (2199/2199), 2.60 MiB | 2.80 MiB/s, done.
Resolving deltas: 100% (1525/1525), done.
Processing /content/joeynmt
     |████████████████████████████████| 266kB 7.9MB/s 
     |████████████████████████████████| 307kB 42.4MB/s 
     |████████████████████████████████| 215kB 44.3MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 737kB 44.4MB/s 
     |████████████████████████████████| 61kB 8.5MB/s 
  Created wheel for joeynmt: filename=joeynmt-0.0.1-cp36-none-any.whl size=72136 sha256=f89096f6dc4b35697dc6609bc1739a621f9e13a0ecf58ad0e963439e575bc81b
  Stored in directory: /tmp/pip-ephem-wheel-cache-49iy698r/wheels/db/01/db/751cc9f3e7f6faec127c43644ba250a3ea7ad200594aeda70a
  Created wheel for pyyaml:

# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [0]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path

! cp bpe.codes.4000 $data_path
! ls $data_path

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

# Some output
! echo "BPE Yoruba Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$src$tgt/vocab.txt  # Herman

bpe.codes.4000	dev.en	     test.bpe.xh     test.xh	   train.en
dev.bpe.en	dev.xh	     test.en	     train.bpe.en  train.xh
dev.bpe.xh	test.bpe.en  test.en-any.en  train.bpe.xh
bpe.codes.4000	dev.en	     test.bpe.xh     test.xh	   train.en
dev.bpe.en	dev.xh	     test.en	     train.bpe.en  train.xh
dev.bpe.xh	test.bpe.en  test.en-any.en  train.bpe.xh
BPE Yoruba Sentences
Oku kwa@@ ph@@ umela ekubeni nd@@ id@@ ume njengom@@ ntu ong@@ any@@ anis@@ ekanga .
Xa nd@@ afunda inyaniso , and@@ izange nd@@ iph@@ inde nd@@ iv@@ ume ukuq@@ hubeka ndis@@ enza oko , naku@@ beni ndand@@ ihl@@ awul@@ wa um@@ v@@ uzo on@@ c@@ um@@ isayo .
Nd@@ ing@@ umzekelo om@@ hle ko@@ onyana bam abab@@ ini yaye nd@@ iye nd@@ af@@ anel@@ ekela amalung@@ elo ang@@ akumbi ebandleni .
Nd@@ az@@ iwa njengom@@ ntu on@@ yanisekileyo ngabantu end@@ ish@@ ish@@ ina kunye nabo kwan@@ abahl@@ ol@@ i boku@@ m@@ a kwem@@ ali ye@@ enk@@ amp@@ ani ( t@@ a@@ x a@@ ud@@ it@@ ors ) . ”
U@@ R@@ ute wa@@ f@@ ud@@ ukela kwa@@ Sirayeli ap

In [0]:
# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

bpe.codes.4000	dev.en	     test.bpe.xh     test.xh	   train.en
dev.bpe.en	dev.xh	     test.en	     train.bpe.en  train.xh
dev.bpe.xh	test.bpe.en  test.en-any.en  train.bpe.xh


# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [0]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 50                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

#*Tensorboard*
JoeyNMT additionally uses TensorboardX to visualize training and validation curves and attention matrices during training. Launch Tensorboard (requires installation that is not included in JoeyNMTs requirements) like this:

In [0]:
# Restart runtime using 'Runtime' -> 'Restart runtime...'
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

1.15.0


In [0]:
%load_ext tensorboard

In [0]:
%tensorboard --logdir "$gdrive_path/models/enxh_transformer/tensorboard"

# Train the Model

This single line of joeynmt runs the training using the config we made above

In [0]:
# # Train the model
# # You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
# !cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

In [0]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

2019-11-28 03:28:33,220 Hello! This is Joey-NMT.
2019-11-28 03:28:36,730 Total params: 12201984
2019-11-28 03:28:36,731 Trainable parameters: ['decoder.layer_norm.bias', 'decoder.layer_norm.weight', 'decoder.layers.0.dec_layer_norm.bias', 'decoder.layers.0.dec_layer_norm.weight', 'decoder.layers.0.feed_forward.layer_norm.bias', 'decoder.layers.0.feed_forward.layer_norm.weight', 'decoder.layers.0.feed_forward.pwff_layer.0.bias', 'decoder.layers.0.feed_forward.pwff_layer.0.weight', 'decoder.layers.0.feed_forward.pwff_layer.3.bias', 'decoder.layers.0.feed_forward.pwff_layer.3.weight', 'decoder.layers.0.src_trg_att.k_layer.bias', 'decoder.layers.0.src_trg_att.k_layer.weight', 'decoder.layers.0.src_trg_att.output_layer.bias', 'decoder.layers.0.src_trg_att.output_layer.weight', 'decoder.layers.0.src_trg_att.q_layer.bias', 'decoder.layers.0.src_trg_att.q_layer.weight', 'decoder.layers.0.src_trg_att.v_layer.bias', 'decoder.layers.0.src_trg_att.v_layer.weight', 'decoder.layers.0.trg_trg_att.k_l

In [0]:
# Copy the created models from the notebook storage to google drive for persistant storage 
!mkdir -p "$gdrive_path/models/${src}${tgt}_transformer/" # Herman
!cp -r joeynmt/models/${src}${tgt}_transformer/* "$gdrive_path/models/${src}${tgt}_transformer/"



In [0]:
# Output our validation accuracy
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

In [0]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${src}${tgt}_transformer/config.yaml"

# Plot Perplexity and Bleu

In [0]:
#Plot Perplexity
! python3 joeynmt/scripts/plot_validations.py "$gdrive_path/models/${src}${tgt}_transformer" \
--plot_values PPL \
--output_path "$gdrive_path/models/${src}${tgt}_transformer/ppl.png"

# from IPython.display import Image
from IPython.display import Image, display
display(Image("$gdrive_path/models/${src}${tgt}_transformer/ppl.png"))

In [0]:
#Plot Bleu Score 
! python3 joeynmt/scripts/plot_validations.py "$gdrive_path/models/${src}${tgt}_transformer" \
--plot_values bleu \
--output_path "$gdrive_path/models/${src}${tgt}_transformer"/bleu.png

# from IPython.display import Image
from IPython.display import Image, display
display(Image("$gdrive_path/models/${src}${tgt}_transformer/bleu.png"))

# Copy model from virtual drive

In [0]:
#Remove Afterwards
# !cp -r "/content/drive/My Drive/masakhane/en-yo-baseline/" /content/

# NMT Attention Alignment Visualizations

In [0]:
# # Install keras attention
# ! cd content;
# ! git clone https://github.com/thushv89/attention_keras.git
# ! cd /content/attention_keras;

In [0]:
# ! cd /content;
# ! git clone https://github.com/M4t1ss/SoftAlignments.git

https://github.com/zhaocq-nlp/Attention-Visualization
https://github.com/shreydesai/attention-viz

##vizseq

In [0]:
! pip install vizseq

In [0]:
! pip install -U tqdm

In [0]:
! pip3 install -U nltk

In [0]:
import vizseq

In [0]:
from glob import glob
root = '/content/drive/My Drive/Colab Notebooks/MyJoeyNMT'
src, ref, hypo = glob(f'{root}/navy_xhen/word/test.xh'), glob(f'{root}/navy_xhen/word/test.en'), glob(f'{root}/models/transformer_xhen/predictions.test')

In [0]:
#First, load the vizseq package:
vizseq.view_stats(src, ref)

In [0]:
vizseq.view_n_grams(src)

In [0]:
# To view corpus-level scores (BLEU and METEOR):
vizseq.view_scores(ref, hypo, ['bleu', 'meteor'])

In [0]:
vizseq.available_scorers()

In [0]:
# import vizseq.VizSeqSortingType
vizseq.view_examples(src, ref, hypo, ['bleu', 'meteor'], page_sz=2, page_no=12)

In [0]:
# Google Translate Integration
vizseq.set_google_credential_path('path to google credential json file')

In [0]:
vizseq.view_examples(src, ref, hypo, ['bleu'], need_g_translate=True)

In [0]:
from vizseq.ipynb import fairseq_viz as vizseq_fs
log_path = 'examples/data/wmt14_fr_en_test.fairseq_generate.log'
vizseq_fs.view_stats(log_path)
vizseq_fs.view_examples(log_path, ['bleu', 'meteor'], need_g_translate=True)
vizseq_fs.view_scores(log_path, ['bleu', 'meteor'])
vizseq_fs.view_n_grams(log_path)

In [0]:
https://docs.dgl.ai/en/latest/tutorials/models/4_old_wines/7_transformer.html

In [0]:
https://ronxin.github.io/wevi/
https://vcg.seas.harvard.edu/publications/seq2seq-vis-a-visual-debugging-tool-for-sequence-to-sequence-models